In [83]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
import numpy as np

In [84]:
output_folder_path = "../outputs/RealWorld/facebookMedium/bots/01"
input_folder_path="../../output/RealWorld/facebookMedium/bots/01"
#datasets=['astroPh', 'emailEU', 'facebook', 'facebookMedium', 'facebookArtist', 'gitInteractions', 'hepPh', 'lastfm', 'redditHyperlink', 'twitchDE', 'twitchENGB', 'twitchES', 'twitchFR', 'twitchPTBR', 'twitchRU']
#botRatio=['m0_', 'm25_', 'm50_', 'm75_', 'm100_', 'm125_', 'm150_', 'm175_', 'm200_']
datasets=['facebookMedium']
startRatio=['01', '02', '03', '04', '05']
rewireRatio=[ 'r10_', 'r20_', 'r30_', 'r40_', 'r50_', 'r60_', 'r70_', 'r80_', 'r90_']
data2={}
average={}

In [85]:
def create_data(path):
    files=os.listdir(path)

    botRatio=[]
    for file in files:
        if file.endswith(".txt"):
            match = re.search(r"_m(\d+)_", file)
        if match:
            result = match.group(0)
        if not result in botRatio:
            botRatio.append(result)
    botRatio=sorted(botRatio, key=lambda x: int(x.strip('_m')))

    for mat in botRatio:
        data2[mat]={}
        average[mat]={}
        for rr in rewireRatio:
            data2[mat][rr]=[]
            for file in files:
                if mat in file and rr in file:
                    #print(path+" "+file)
                    with open(os.path.join(path, file), 'r') as f:
                        content = pd.read_csv(path+"/"+file, names=['Epoch', 'Pop', 'Frac', 'DiscEdge'], skiprows=1, sep=" ")

                        content=content.drop(['Epoch', 'Pop'], axis=1)    
                        finalPerc=content.iloc[-1, 1]
                        finalEpoch=content.iloc[-1, 0]
                        data2[mat][rr].append(finalEpoch)
            avg=sum(data2[mat][rr])/len(data2[mat][rr])
            if mat not in average: 
                average[mat] = {}
            average[mat][rr]=avg    
        

In [86]:
def create_graphs():
    # Create a 3x3 grid of subplots
    fig, axes = plt.subplots(4, 3, figsize=(15, 15))
    axes = axes.flatten()  # Flatten to iterate over each subplot
    for idx, (outer_key, inner_dict) in enumerate(average.items()):
        x_values = [float(k.strip('_r'))/100 for k in inner_dict.keys()]  # Extract numeric parts from keys
        ok=outer_key
        y_values = list(inner_dict.values())  # Get corresponding values
        x = np.array(x_values)
        y = np.array(y_values)
        bot_ratio=float(outer_key.strip('_m'))/1000

        ax = axes[idx]  # Select the subplot
        outer_key=''.join(filter(str.isdigit, outer_key))
        ax.plot(x, y, marker='o', label=outer_key)
        print(ok)
        for rewiring_key, values_list in data2[ok].items():
            x_val = float(rewiring_key.strip('_r')) / 100
            y_vals = values_list  # List of values for this rewiring rate                
            ax.scatter([x_val] * len(y_vals), y_vals, color='red', alpha=0.5, label="Data points" if idx == 0 else "")

        ax.set_title(f'Final Op. Density Vs Rewiring Rate for BotRatio = {bot_ratio}')
        ax.set_xlabel('Rewiring Rate')
        ax.set_ylabel('Final Opinion Density of Minority')
        ax.grid(True)
    # Hide any unused subplots if data has fewer than 9 keys
    for idx in range(len(average), 9):
        fig.delaxes(axes[idx])

    # Adjust layout
    plt.tight_layout()
    plt.show()
    figpath=output_folder_path+"/MinorityOpinionVsRewiring.png"
    fig.savefig(figpath, dpi=350)
        

In [1]:
create_data(input_folder_path)
create_graphs()

NameError: name 'create_data' is not defined